In [1]:
import multiprocessing
import time

import sys
sys.path.append('../python/')
import cfg
import db
import fio
import hmt
import lex

In [2]:
corpus_id = cfg.CORPUS_ID_HMT

In [3]:
# initialize the database, extract meta-data from logs/transcriptions
db.connect(corpus_id)
# careful, this DELETES ALL DB TABLES
db.executescript(cfg.SQL_PATH, cfg.SQL_INIT_FNAME_HMT)
db.commit()

hmt.populate_speakers()
hmt.populate_sessions()
hmt.populate_tasks()
hmt.populate_turns_and_chunks()
db.set_turn_index_ses()
db.commit()
db.close()

In [4]:
# extract features for all chunks
# (takes about an hour on my machine)

pool = multiprocessing.Pool(7)

db.connect(corpus_id)
task_lst = db.get_tasks()
db.close()

start = 0
step = 14

for i in range(start, len(task_lst), step):
    print('%d %s\n' % (i, time.ctime()))
    args = [task_lst[j] for j in range(i, min(i+step, len(task_lst)))]
    results = pool.starmap(hmt.extract_features, args)
print('done! %s\n' % time.ctime())

0 Thu Apr  7 14:49:56 2022

14 Thu Apr  7 15:11:35 2022

28 Thu Apr  7 15:29:17 2022

done! Thu Apr  7 15:41:06 2022



In [5]:
# run cleanup (set all features null for all chunks with any null)
db.connect(corpus_id)
db.executescript(cfg.SQL_PATH, cfg.SQL_CU_FNAME)
db.commit()
db.close()

In [6]:
# make timestamps continuous per session (run this only once!)
db.connect(corpus_id)
db.executescript(cfg.SQL_PATH, cfg.SQL_FT_FNAME)
db.commit()
db.close()

In [7]:
# create auxiliary table chunk_pairs
db.connect(corpus_id)
db.executescript(cfg.SQL_PATH, cfg.SQL_AT_FNAME)
db.commit()
db.close()

In [8]:
# compute auxiliary files for lex measures
db.connect(corpus_id)
# load default lemmatizer
_, _, lem = lex.load_lex_w_lem(cfg.LEX_LEM_FNAME)
# store lemmatized tokens & language models for sessions and tasks 
# (tasks have sufficient length to be considered separately)
fio.store_tokens(corpus_id, None, lem)
lex.store_lms_ngrams(corpus_id)
db.close()